# 自动求导

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。

In [14]:
import torch
x=torch.arange(4.0)#生成一个关于x的向量

In [15]:
help(torch.dot)

Help on built-in function dot in module torch:

dot(...)
    dot(input, other, *, out=None) -> Tensor
    
    Computes the dot product of two 1D tensors.
    
    .. note::
    
        Unlike NumPy's dot, torch.dot intentionally only supports computing the dot product
        of two 1D tensors with the same number of elements.
    
    Args:
        input (Tensor): first tensor in the dot product, must be 1D.
        other (Tensor): second tensor in the dot product, must be 1D.
    
    Keyword args:
        out (Tensor, optional): the output tensor.
    
    Example::
    
        >>> torch.dot(torch.tensor([2, 3]), torch.tensor([2, 1]))
        tensor(7)



In [16]:
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度。**]

注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。

In [17]:
x.requires_grad_(True) # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad                   # 默认值是None

(**现在来计算$y$.此时$y=2\mathbf{x}^{\top}\mathbf{x}$**)


In [18]:
y = 2 * torch.dot(x, x)  #dot表示向量点乘，及两个向量元素相乘并相加
y ## （0*0+1*1+2*2+3*3）*2=28

tensor(28., grad_fn=<MulBackward0>)

`x`是一个长度为4的向量，计算`x`和`x`的点积，得到了我们赋值给`y`的标量输出。
接下来，[**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**]，并打印这些梯度。

**Pytorch中的.backward()方法**
https://blog.csdn.net/deephub/article/details/115438881?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522171319196216800180649477%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=171319196216800180649477&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-115438881-null-null.142^v100^pc_search_result_base9&utm_term=backword%E6%96%B9%E6%B3%95&spm=1018.2226.3001.4187

In [19]:
y.backward()##对y利用反向传播函数，对x求偏导
x.grad

tensor([ 0.,  4.,  8., 12.])

In [20]:
x.grad == 4 * x

tensor([True, True, True, True])

In [35]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()##把x.grad数据清空为0
y = x.sum()##重新计算y
y.backward()
x.grad##1111

tensor([1., 1., 1., 1.])

In [36]:
y.backward()
x.grad##1111

tensor([2., 2., 2., 2.])

## 分离计算

有时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

这里可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。

In [33]:
x.grad.zero_()
y = x * x


##y.backward()  不可以运行
##x.grad        不可以运行


u = y.detach()##detach 把y的值看作一个常数对待
z = u * x
z.sum().backward()
x.grad == u


y


tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。

In [34]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [37]:
def f(a):
    b = a * 2
    while b.norm() < 1000: ## norm表示求范式，也就是欧几里得距离。默认是二阶范式
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [45]:
a = torch.randn(size=(), requires_grad=True)##randn表示随机数，size=0表示标量%alia

In [51]:


d = f(a)##d永远是关于a的线性函数
a

tensor(-0.0340, requires_grad=True)

In [52]:
d

tensor(-111433.9688, grad_fn=<MulBackward0>)

In [53]:
d.backward()

In [54]:
a.grad

tensor(6553600.)

In [55]:
d/a

tensor(3276800., grad_fn=<DivBackward0>)

In [56]:
a.grad == d / a

tensor(False)

## 小结

* 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上，然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

**x=torch.arange(4.0)**                #生成一个关于x的向量

**x.requires_grad_(True)**              # 等价于x=torch.arange(4.0,requires_grad=True)

**x.grad**                          # 默认值是None

**y = 2 * torch.dot(x, x)**  #dot表示向量点乘，及两个向量元素相乘并相加

**u = y.detach()**##detach 把y的值看作一个常数对待

**x.grad.zero_()**##把x.grad数据清空为0

**a = torch.randn(size=(), requires_grad=True)**##randn表示随机数，size=()表示标量


## 练习

1. 为什么计算二阶导数比一阶导数的开销要更大？
1. 在运行反向传播函数之后，立即再次运行它，看看会发生什么。
1. 在控制流的例子中，我们计算`d`关于`a`的导数，如果将变量`a`更改为随机向量或矩阵，会发生什么？
1. 重新设计一个求控制流梯度的例子，运行并分析结果。
1. 使$f(x)=\sin(x)$，绘制$f(x)$和$\frac{df(x)}{dx}$的图像，其中后者不使用$f'(x)=\cos(x)$。
